In [1]:
import pandas as pd
import numpy as np
import math
import random

In [2]:
#define parameters (ori)

pop_size = 50
maxIter = 1000
lb=[-5,-5]
ub=[5,5]

In [3]:
#define initial population

def initial_population(lb, ub, pop_size):
    pop = []
    
    for i in range(pop_size):
        p = []
        for a,b in zip(lb, ub):
            p.append((b - a)*random.random() + a)
        pop.append(p)
    init_pop = pd.DataFrame(pop)
#     print('Dataframe of initial population (5 first rows):')
#     print(init_pop.head())
    
    return init_pop

In [4]:
#define objective function (ori)

def objective_function(p1):
    F = []
    
    for i in range(len(p1)):
        x = p1.loc[i]
        f = (((x[0]**2) + x[1] - 11)**2) + ((x[0] + x[1]**2) - 7)**2
        F.append(f)
    return F    

In [5]:
#update the initial position of the population

def update_population(p1, dim):
    best_x = np.array(p1.loc[p1['f'].idxmin()][0:dim])
    worst_x = np.array(p1.loc[p1['f'].idxmax()][0:dim])
    
    new_x = []
    
    for i in range(len(p1)):
        old_x = np.array(p1.loc[i][0:dim])
        r1 = np.random.random(dim)
        r2 = np.random.random(dim)
        
        new_x.append(old_x + r1*(best_x - abs(old_x)) - r2*(worst_x - abs(old_x)))
    new_x = pd.DataFrame(new_x)
#     print('Updated population (5 first rows): ')
#     print(new_x.head())
    return new_x

In [6]:
#compare - replace the particle position related to the fitness value

def compare_replace(p1, new_p1):
    for i in range(len(p1)):
        if p1.loc[i]['f'] > new_p1.loc[i]['f']:
            p1.loc[i] = new_p1.loc[i]
    return p1       

In [7]:
#maintain the position of the particle

def maintain_position(new_p1, lb, ub):
    for i in range(len(new_p1)):
        for j in range(len(new_p1.columns)-1):
            if new_p1.iloc[i, j] > ub[j]:
                new_p1.iloc[i, j] = ub[j]
            elif new_p1.iloc[i, j] < lb[j]:
                new_p1.iloc[i, j] = lb[j]
                
    return new_p1

In [8]:
#main jaya function

def jaya(*argv):
    pop_size, maxIter, lb, ub = argv
    
    lb = np.array(lb)
    ub = np.array(ub)
    
    p1 = initial_population(lb, ub, pop_size)
    p1['f'] = objective_function(p1)
    
    dim = len(lb)
    Iter = 0
    
    best = []
    
    
    while Iter < maxIter:
        new_p1 = update_population(p1, dim)
        new_p1 = maintain_position(p1, lb, ub)
        new_p1['f'] = objective_function(new_p1)
        
        p1 = compare_replace(p1, new_p1)
        Iter = Iter+1
        
        best = p1['f'].min()
        x_best = p1.loc[p1['f'].idxmin()][0:dim].tolist()
    return best, x_best    

In [9]:
%%time


best,x_best = jaya(pop_size, maxIter, lb, ub)
print('Best fitness value = {}'.format(best))
print('x best as the best position of particle = {}'.format(x_best))

Best fitness value = 5.940686505385222
x best as the best position of particle = [-3.7531961545366856, -2.8851641368135827]
CPU times: total: 1min 9s
Wall time: 1min 11s
